In [ ]:
!pip install requests seaborn

In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

API_KEY = "your_api_key_here"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
FORECAST_URL = "http://api.openweathermap.org/data/2.5/forecast"

weather_history = []

In [ ]:
def get_weather(city):
    params = {"q": city, "appid": API_KEY, "units": "metric", "lang": "ru"}
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_data = {
            "Город": data["name"],
            "Температура": data["main"]["temp"],
            "Описание": data["weather"][0]["description"],
            "Время": datetime.now()
        }
        weather_history.append(weather_data)
        print(f"{weather_data['Город']}: {weather_data['Температура']}°C, {weather_data['Описание']}")
    else:
        print("Ошибка при получении данных.")

In [ ]:
def plot_weather():
    if not weather_history:
        print("Нет данных для отображения.")
        return

    df = pd.DataFrame(weather_history)
    plt.figure(figsize=(10, 6))
    sns.lineplot(x="Время", y="Температура", data=df, marker="o")
    plt.title("Изменение температуры")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def save_to_csv(filename="weather_history.csv"):
    if not weather_history:
        print("Нет данных для сохранения.")
        return
    df = pd.DataFrame(weather_history)
    df.to_csv(filename, index=False)
    print(f"Сохранено в {filename}")

In [ ]:
def get_forecast(city):
    params = {"q": city, "appid": API_KEY, "units": "metric", "lang": "ru"}
    response = requests.get(FORECAST_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        forecast_list = data["list"]
        forecast_df = pd.DataFrame([{
            "Время": datetime.fromtimestamp(item["dt"]),
            "Температура": item["main"]["temp"],
            "Описание": item["weather"][0]["description"]
        } for item in forecast_list])

        plt.figure(figsize=(12, 6))
        sns.lineplot(x="Время", y="Температура", data=forecast_df, marker="o")
        plt.title(f"Прогноз погоды: {city}")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    else:
        print("Не удалось получить прогноз.")

In [ ]:
# Пример использования:
get_weather("Москва")
get_weather("Краков")
plot_weather()
save_to_csv()
get_forecast("Москва")